# CSC324 Lecture 6: Monads

Last lecture, we talked about functors (`fmap`, mappings of the form `f a (a -> b) -> f b`) and applicative functors (`<*>`, mappings of the form `f a -> f (a -> b) -> f b`). Today, we will talk about `Monads`  

## Monads

### Motivation

Consider the following function below. 

In [1]:
import Data.Maybe as Maybe
-- monadicFunction :: a -> Maybe a
monadicFunction x = Just (x + 1)
:t monadicFunction

monadicFunction :: forall a. Num a => a -> Maybe a

Notice that unlike the functions we described in previous lectures, the monadicFunction takes a normal number and returns a data type (`Maybe` in the above example).

In [2]:
monadicFunction 3
b = monadicFunction 5
:t b

Just 4

b :: forall a. Num a => Maybe a

In [3]:
applyMaybe (Just 3) monadicFunction

<interactive>:1:1: error: Variable not in scope: applyMaybe :: Maybe Integer -> (Integer -> Maybe Integer) -> t

Now suppose I have a value, but it is wrapped in a `Just`. That is suppose, I have `Just 14` and I want to apply the monadicFunction to get `Just 15`. The issue is that I can't just pass in the `Just 14` as the function takes the number `14` instead of the entire `Maybe`, `Just 14`.

In [4]:
monadicFunction (Just 3) 

<interactive>:1:1: error:
    • Non type-variable argument in the constraint: Num (Maybe a)
      (Use FlexibleContexts to permit this)
    • When checking the inferred type
        it :: forall a. (Num a, Num (Maybe a)) => Maybe (Maybe a)

Let's define a function that can handle this 

In [5]:
applyMaybe :: Maybe a -> (a -> Maybe b) -> Maybe b   -- Notice the signature is exactly what we want
-- The second argument is of the type similar to the monadicFunction we described above
applyMaybe Nothing f = Nothing -- We don't do anything on Nothing
applyMaybe (Just x) f = f x -- We apply f on that value

In [6]:
applyMaybe (Just 3) monadicFunction

Just 4

Yaay that worked! We essentially uncovered what monads are. Similar to functors and applicative functors, a monad is a typeclass that has a function (**binder**) that takes in a *Monad* and feeds it to a function that takes in a value and returns a *Monad*  
In the above example, `applyMaybe` is considered to be the **binder** (`>>=` symbol) to the monad.


### The Monad Typeclass

Here is what a `Monad` typeclass looks like:

```
class Monad m where
    return :: a -> m a
    
    (>>=) :: m a -> (a -> m b) -> m b
    (>>) :: m a -> m b -> m b
    
    x >> y = x >>= \_ -> y 
    
    fail :: String -> m a
    fail msg = error msg
    

```

Notice that `return` is very similar to `pure` in Functors. As a matter of fact, they do the same thing. The functions `>>` and `fail` are implemented defaultly by Haskell and as long as you have `pure`, `return` is also implemented by default. Thus, you must have an instance of `Applicative Functor` before writing for `Monad` class. The main function that you have to implement is `>>=` aka the bind function. Here's an example to the `Maybe` class

In [7]:
data OurMaybe x = OurNothing | OurJust x

In [8]:
instance Functor OurMaybe where
    fmap f OurNothing = OurNothing
    fmap f (OurJust x) = OurJust (f x)

instance Applicative OurMaybe where
    pure = OurJust
    b <*> OurNothing = OurNothing
    (OurJust f) <*> (OurJust x) = OurJust (f x)

In [9]:
instance Monad OurMaybe where
    return = OurJust
    OurNothing >>= f = OurNothing
    (OurJust x) >>= f = f x

### Why Monads ?
Monads are very powerful. It may seem like applicative functors are stronger than monads, since applicative functors allow us to take a normal function and make it operate on values with contexts. We'll see that monads can do that as well because they're an upgrade of applicative functors, and that they can also do some cool stuff that applicative functors can't.